In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
import numpy as np

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm

In [4]:
AKSES = 'data/kemudahan_akses_ntb.geojson'
FASKES = 'data/puskesmas_dokter_ntb.geojson'
SLOPE = 'data/centroid_slope_grid.geojson'
ELEVATION = 'data/centroid_elevation_grid.geojson'
RIVER = 'data/osm_ntb_river.geojson'
RESULT = 'data/new_facility_analysis_rs.geojson'

In [5]:
akses = gpd.read_file(AKSES)
slope = gpd.read_file(SLOPE)
elevation = gpd.read_file(ELEVATION)
river = gpd.read_file(RIVER)
faskes = gpd.read_file(FASKES)

In [7]:
df = gpd.sjoin(akses, slope[['Slope1', 'geometry']])
df = df.drop('index_right', axis=1)
df = gpd.sjoin(df, elevation[['Elevation1', 'geometry']])
df = df.drop('index_right', axis=1)

In [8]:
df.loc[df['Slope1'].isnull()==True, 'Slope1'] = df['Slope1'].min()
df.loc[df['Elevation1'].isnull()==True, 'Elevation1'] = df['Elevation1'].min()

In [9]:
river = river.to_crs(epsg='3857')
river['geometry'] = river['geometry'].buffer(distance=100)

In [10]:
df = gpd.sjoin(df, river[['waterway', 'geometry']], how='left')
df = df.drop('index_right', axis=1)
df = df.drop_duplicates('id')

In [11]:
df.loc[df['waterway'].isnull()==True, 'waterway'] = 'land'

In [12]:
faskes = faskes.to_crs(epsg='3857')
faskes['geometry'] = faskes['geometry'].buffer(distance=5000)

In [13]:
faskes.loc[:, 'facility'] = 'available'

In [14]:
df = gpd.sjoin(df, faskes[['facility', 'geometry']], how='left')
df = df.drop('index_right', axis=1)
df = df.drop_duplicates('id')

In [15]:
df.loc[df['facility'].isnull()==True, 'facility'] = 'unavailable'

In [16]:
df = df.loc[
    (df['facility']=='unavailable')
    & (df['waterway']=='land')
    & (df['Slope1']<15)
    & (df['Access_Label']=='Bad')
]

In [55]:
df.to_file(RESULT)